In [57]:
#Create a list of viruses with unique family, genus, tnumber, genome combinations
# current code removes only_ca structures, see comments to just deprioritize them
# resolution breaks ties
#Uses res.txt which stores resolution info and writes to uniq_lres.xlsx, id_fold.txt containing fold data
# and only_ca.txt containing ids with only c alpha atoms (see analyze_aas.ipynb for details)

with open('data/only_ca.txt', 'r') as f:
    only_ca = [l.replace('\n', '') for l in f]

import json
with open('data/db.json', 'r') as f:
    data = json.load(f)


fams = data['filters']['family']

gens = data['filters']['genus']

tnums = data['filters']['tnumber']

genomes = data['filters']['genome']

reslist = []

filters = {'family': fams, 'genus': gens,'tnumber': tnums,'genome': genomes}



ids = []
el_lists = []
for k in data['data']:
    el = data['data'][k]
    if 'family' in el.keys() and 'genome' in el.keys() and 'genus' in el.keys() and 'tnumber' in el.keys():
        el_lists.append([el['family'], el['genus'], el['tnumber'], el['genome'], k])
        ids.append(k)
        
with open('data/res.txt', 'r') as f:
    id_res = {l.split()[0]:float(l.split()[1]) for l in f}
for k in ids:
    if k not in id_res:
        id_res[k] = 9999
    
el_lists_red = [e[:4] for e in el_lists]

unique_combs = [t.split(',') for t in list(set([','.join(e) for e in el_lists_red]))]

with open('data/id_fold.txt', 'r') as f:
    id_fold = {l.split()[0]:l.split(' ',1)[1].replace('\n', '') for l in f}

def find_best(els):
    non_ca = [e for e in els if e[2] not in only_ca]
    if not len(non_ca):
        return []
    els = non_ca
    '''
    uncomment this and comment above if statement to get deprioritized
    if len(non_ca):
        els = non_ca
    '''
    resdict = {e[0]:e for e in els}
    minres = min(resdict.keys())
    return resdict[minres]
res_id_tups = [[id_res[i[-1]],i[:4],i[-1]] for i in el_lists]
lres = []
unids = []
for u in unique_combs:
    matching = [x for x in res_id_tups if x[1] == u]
    best = find_best(matching)
    if best == []: continue
    fold = id_fold[best[2]] if best[2] in id_fold else ''
    lres.append([best[0]] + best[1] + [best[2]] + [fold])
    unids.append(best[2])

In [43]:
columns = ['resolution', 'family', 'genus', 'tnumber', 'genome', 'id', 'fold']

dfdict = {}

for i in range((len(columns))):
    es = [e[i] for e in lres]
    dfdict[columns[i]] = es

In [44]:
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd

# xl_file = 'data/uniq_lres_ca_deprioritized.xlsx'
xl_file = 'data/uniq_lres_ca_removed.xlsx'
df = pd.DataFrame(dfdict)
df.to_excel(xl_file, index = False)

In [46]:
with open('data/unids_ca.txt', 'w') as f:
    for i in unids:
        f.write(i + '\n')

In [63]:
'''
used to write the ids in the deprioritized list not in the excluded list to uniq_only_ca.txt
with open('data/unids_ca.txt', 'r') as f:
    unids_exc = [l.replace('\n', '') for l in f]
ext = [e for e in unids if e not in unids_exc]
with open('data/uniq_only_ca.txt', 'w') as f:
    for i in ext:
        f.write(i + '\n')
'''